<center>
  <h1>Digital Tools and Methods for the Humanities and Social Sciences</h1>
  <img src="https://raw.githubusercontent.com/sul-cidr/Workshops/master/cidr-logo.no-text.240x140.png" alt="Center for Interdisciplinary Digital Research @ Stanford"/>
</center>

# Data Manipulation with Python and Pandas

### Instructors
- Simon Wiles (CIDR), <em>simon.wiles@stanford.edu</em>
- Peter Broadwell (CIDR), <em>broadwell@stanford.edu</em>

### Sign In
Please sign in for this workshop at: https://signin.cidr.link/Python_and_Pandas/ -- when you've submitted the sign-in form, please keep your browser tab open on the evaluation form as a reminder to complete it when the workshop is over.


### Goal
By the end of this workshop, we hope you'll be able to load in data into a Pandas `DataFrame`, perform basic cleaning and analysis, and create visualizations of some relevant aspects of a dataset.  For most of this workshop we will work with a dataset prepared from the [IMDb Datasets](https://www.imdb.com/interfaces/) and the [OMDb API](https://www.omdbapi.com/).

### Topics
- What is Pandas?
  - What does Pandas do?
  - Where can I get more help with Pandas?
- Introduction to `DataFrame`s and `Series`
- Creating `DataFrame`s and loading data
  - Creating `DataFrame`s from data
  - Reading data from persistent storage
  - Writing `DataFrames` back out to persistent storage
- Working with `DataFrames`
  - Exploring `DataFrames`
  - Slicing and sub-setting
  - Operations, filtering, and assignment
  - Cleaning and manipulating `DataFrames`
  - Aggregating data
- Visualization

### Evaluation survey
At the end of the workshop, we would be very grateful if you can, please, spend 1 minute answering a few questions that will help us to continue our workshop series.
- https://evaluations.cidr.link/Python_and_Pandas/


---

### Jupyter Notebooks and Google Colaboratory

Jupyter notebooks are a way to write and run Python code in an interactive way. They're quickly becoming a standard way of putting together data, code, and written explanations or visualizations into a single document and sharing that. There are a lot of ways that you can run Jupyter notebooks, including just locally on your computer, but we've decided to use Google's Colaboratory notebook platform for this workshop.  Colaboratory is “a Google research project created to help disseminate machine learning education and research.”  If you would like to know more about Colaboratory in general, you can visit the [Welcome Notebook](https://colab.research.google.com/notebooks/welcome.ipynb).

Using the Google Colaboratory platform allows us to focus on learning and writing Python in the workshop rather than on setting up Python, which can sometimes take a bit of extra work depending on platforms, operating systems, and other installed applications. If you'd like to install a Python distribution locally, though, we have some instructions (with gifs!) on installing Python through the Anaconda distribution, which will also help you handle virtual environments: https://github.com/sul-cidr/Workshops/wiki/Installing-and-Configuring-Anaconda-and-Jupyter-Notebooks

If you run into problems, or would like to look into other ways of installing Python or handling virtual environments, feel free to send us an email (contact-cidr@stanford.edu).

### Environment
If you would prefer to use Anaconda or your own local installation of python or Jupyter Notebooks, for this workshop you will need an environment with the following packages installed and available:
- `pandas`
- `matplotlib`
- `sqlalchemy`

Please note that we will likely not have time during the workshop to support you with problems related to a local environment, and we do recommend using the Colaboratory notebooks if you are at all unsure.

---

## 1. What is Pandas?

Pandas is a high-level data manipulation tool first created in 2008 by Wes McKinney.  The name is derived from the term “panel data,” an econometrics term for data sets that include observations over multiple time periods for the same individuals.<sup>[[wikipedia](https://en.wikipedia.org/wiki/Pandas_(software))]</sup>

From Jake Vanderplas’ book [**Python Data Science Handbook**](http://shop.oreilly.com/product/0636920034919.do):

> Pandas is a newer package built on top of NumPy, and provides an efficient implementation of a `DataFrame`. `DataFrame`s are essentially multidimensional arrays with attached row and column labels, and often with heterogeneous types and/or missing data. As well as offering a convenient storage interface for labeled data, Pandas implements a number of powerful data operations familiar to users of both database frameworks and spreadsheet programs.

In [ ]:
import pandas as pd

# The two lines below configure how our outputs are shown in this notebook
#  environment.  They need not concern us now.
pd.set_option('display.max_rows', 20)
pd.DataFrame._repr_html_ = \
    lambda self: ('<style>table.dataframe td {white-space: nowrap}</style>' +
                  self.to_html(max_rows=10, show_dimensions=True, notebook=True))

### 1.1. What does Pandas *do*?

* Reading and writing data from persistent storage
* Cleaning, filtering, and otherwise preparing data
* Calculating statistics and analyzing data
* Visualization with help from Matplotlib

... but perhaps we should let Pandas introduce itself:

In [ ]:
pd?

### 1.2. Where can I get more help with Pandas?

The [Pandas website](https://pandas.pydata.org/) and [online documentation](http://pandas.pydata.org/pandas-docs/stable/) are useful resources, and of course the indispensible [Stack Overflow has a "pandas" tag](https://stackoverflow.com/questions/tagged/pandas).  There is also a (much younger, much smaller) [sister site dedicated to Data Science questions that has a "pandas" tag](https://datascience.stackexchange.com/questions/tagged/pandas) too.

In [ ]:
pd.isnull?

## 2. Introduction to `DataFrame`  s and `Series`

The main data structure that Pandas implements is the `DataFrame`, and a `DataFrame` is composed of one or more `Series` and, optionally, an `Index`.  

A `DataFrame` is a two-dimensional array with flexible row indices and flexible column names. It can be thought of as a generalization of a two-dimensional NumPy array, or a specialization of a dictionary in which each column name maps to a `Series` of column data.

A `Series` is a one-dimensional array of indexed data. It can be thought of as a specialized dictionary or a generalized NumPy array.

A `DataFrame` is made up of `Series` in a similar way in which a table is made up of columns. The only restriction is that each column must be of the same data type.  Many of the operations that can be performed on a `DataFrame` can also be performed on an individual `Series`.


<img src="https://raw.githubusercontent.com/sul-cidr/Workshops/master/Data_Manipulation_with_Python/assets/dataframes.png" alt="DataFrames are composed of Series">

## 3. Creating `DataFrame`s and loading data

There are a great many ways to create a Pandas `DataFrame` -- we can build one ourselves in lower-level Python datatypes, of course, but Pandas also provides methods to load data in from common storage and serialization formats.

<a title="PerryPlanet [Public domain], via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Bayarea_map.svg" style="float:right"><img width="256" alt="Bayarea map" src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/78/Bayarea_map.svg/512px-Bayarea_map.svg.png"></a>
### 3.1. Creating `DataFrame`s from data

The simplest way to generate a `DataFrame` is to create it directly from a `dict` of `list`s:

In [ ]:
data = {
    "county": ["Alameda", "Contra Costa", "Marin", "Napa", "San Francisco", "San Mateo", "Santa Clara", "Solano", "Sonoma"],
    "county seat": ["Oakland", "Martinez", "San Rafael", "Napa", "San Francisco", "Redwood City", "San Jose", "Fairfield", "Santa Rosa"],
    "population": [1494876, 1037817, 250666, 135377, 870887, 711622, 1762754, 411620, 478551],
    "area": [2130, 2080, 2140, 2040, 600.59, 1930, 3380, 2350, 4580]
}
bay_area_counties = pd.DataFrame(data)
bay_area_counties

Pandas has automatically created an `Index` on this `DataFrame` ([0..8]), but we can also specify our own `Index` when we instantiate the frame ourselves:

In [ ]:
bay_area_counties = pd.DataFrame(data, index=["Ala", "Con", "Mar", "Nan", "SF", "SM", "SC", "Sol", "Son"])
bay_area_counties

This allows us to `loc`ate a specific reference using the key in the `Index`:

In [ ]:
bay_area_counties.loc['Ala']

We can also set an `Index` at any time after the `DataFrame` has been created, either by adding a new index:

In [ ]:
bay_area_counties = pd.DataFrame(data)
bay_area_counties.index = ["Ala", "Con", "Mar", "Nan", "SF", "SM", "SC", "Sol", "Son"]
bay_area_counties

or by choosing one of the existing columns to become the index:

In [ ]:
bay_area_counties = pd.DataFrame(data)
# note the use of `inplace=True`
bay_area_counties.set_index('county', inplace=True)
bay_area_counties

In [ ]:
bay_area_counties.loc['Santa Clara']

### 3.2. Reading data from persistent storage

However, most of the time we're more likely to be reading data in from an external source of some kind, and Pandas has us well covered here.

First, let's grab some data into our Colaboratory Notebook environment so that we can work with it locally:

In [ ]:
!mkdir -p workshop_data
!wget https://raw.githubusercontent.com/sul-cidr/Workshops/master/Data_Manipulation_with_Python/sample_data/imdb_top_1000.csv -O workshop_data/imdb_top_1000.csv

#### 3.2.1. CSV files
Reading in data from CSV files is as simple as:

In [ ]:
df = pd.read_csv('workshop_data/imdb_top_1000.csv')
df

Notice again that Pandas has created a default `Index` for this `DataFrame` -- we probably want the `imdbID` column to be the `Index`, and we can set that after import, as we did above with the `bay_area_counties` data, or we can specify it when loading the CSV initially:

In [ ]:
df = pd.read_csv('workshop_data/imdb_top_1000.csv', index_col='imdbID')
df.head(4) # the `head` method defaults to five if called without an argument
           # a `tail` method is also available with the same semantics

#### 3.2.2. Reading data from JSON Files

JSON files can be loaded in a similarly straightforward way.

There are two things to note here:

1. the nature of JSON as a file format is such that the `Index` is explicit, and Pandas will set it correctly for us initially.
2. We're loading the data directly over HTTP(S) here -- Pandas `read_...` methods can accept a local file path or a URL, and Pandas will take care of fetching the data for you.

In [ ]:
pd.read_json('https://raw.githubusercontent.com/sul-cidr/Workshops/master/Data_Manipulation_with_Python/sample_data/imdb_top_1000.json')

#### 3.2.3. Reading data via a SQL query

We can also load data from relational databases or other datastores that export a SQL-compatible interface.  For this example we'll download a simple SQLite database file to operate on, but Pandas’ `read_sql*` methods can accept a `connection` object that is predicated on a remote database server if required.

In [ ]:
!pip install sqlalchemy
!mkdir -p workshop_data
!wget https://raw.githubusercontent.com/sul-cidr/Workshops/master/Data_Manipulation_with_Python/sample_data/imdb_top_1000.sqlite -O workshop_data/imdb_top_1000.sqlite

In [ ]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///workshop_data/imdb_top_1000.sqlite", echo=False)
pd.read_sql_query("SELECT * FROM imdb_top_1000;", con=engine, index_col='imdbID')

#### 3.2.3. Other input formats

Pandas also has methods that allow it to read data directly from other formats, including those used by Microsoft Excel, Stata, SAS, and Google Big Query.  More details are available from the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html).

### 3.3. Writing `DataFrames` back out to persistent storage

Pandas makes writing data to persistent storage formats similarly convenient.  Methods are available to write to most of the formats Pandas can read, including all those demonstrated above.

In [ ]:
df.to_csv('workshop_data/imdb_data_2.csv')

## 4. Working with `DataFrame`s

Let's begin by loading our dataset of the top 1,000 ranked films on imDB.

In the same way that it's common to `import pandas as pd`, it's common to use `df` as an identifier for generic `DataFrame`s, especially in tutorials and demos.  For anything other than interactive sessions or throw-away scripts, however, we strongly recommend using good descriptive identifiers for your `DataFrame`s!

In [ ]:
df = pd.read_csv('workshop_data/imdb_top_1000.csv', index_col='imdbID')

### 4.1. Exploring `DataFrame`s

Pandas provides a host of ways to explore our data, which is especially useful when we're getting to know a new dataset or source.

In [ ]:
df.info()

In [ ]:
df.at['tt0111161', 'Title']

In [ ]:
df.iat[0, 0]

Accessing columns can be done using the dot notation, `df.column_name`, or the dictionary notation, `df['column_name']`.  This returns a `Series` object.

In [ ]:
df.Title

In [ ]:
type(df.Title)

Pandas allows us to very easily explore aspects of the dataset with a host of information methods:

In [ ]:
df.Rated

In [ ]:
df.Rated.unique()

In [ ]:
df.Rated.value_counts()

For continuous variables, especially, we can quickly get a good handle on the distribution of the data:

In [ ]:
df.imdbVotes.describe()

In [ ]:
df.imdbVotes.describe().apply("{:.1f}".format)

In [ ]:
df[['Title', 'Director']]

In [ ]:
type(df[['Title', 'Director']])

`Series` object and other subsets of `DataFrame`s preserve the indices of the `DataFrame` from which they are derived, which makes further operations such as merging or columns manipulation possible.

`DataFrame`s are designed to operate at the column level, not at the row level. However, a subset of rows can be returned using the same slice notation that will be familiar from regular Python lists.

In [ ]:
df[10:15]

We can chain these operations together, as long as we remember what we are returning in each link of the chain.

In [ ]:
df[10:15].Production

In [ ]:
df.Production[10:15]

In [ ]:
df[['Production']][10:15]

We briefly saw the `.loc` attribute above which allows selection by “label” -- that is, by value(s) in the `Index`.  `DataFrame`s also expose the `.iloc` attribute, which is for selection by (integer-based) position:

In [ ]:
# .reset_index() returns a copy of the DataFrame with the default Index of simple ordinals.
df[["Title", "Plot"]].reset_index().iloc[2:5]

In [ ]:
df[["Title", "Plot"]].loc['tt7286456':'tt0071562']

Notice how in the `.iloc` example, record #5 is omitted, in usual python slice fashion, but in the `.loc` example, the observation with `imdbID == tt0071562` is *included* in the result.

#### 4.2.1 Copying and sorting `DataFrame`s

`DataFrame`s provide the `.sort_values()` method to allow sorting on multiple columns.  Typically we want to sort our data temporarily at time of output -- either as we’re displaying or saving it:

In [ ]:
df.sort_values('Year', ascending=False)

However, sometimes we actually want to change the way in which the observations are ordered in a `DataFrame` in a way which persists, and we can do this using the `inplace=True` argument to the `.sort_values()` method.

To demonstrate this, we're going to reset the index on the `DataFrame` so that observations are given a simple ordinal index.  Since we don't want to change our `DataFrame` for the following examples, we'll make a `.copy()` to operate on.

In [ ]:
df_copy = df.copy()
df_copy.reset_index(inplace=True)
df_copy.head(5)

In [ ]:
df_copy.sort_values(['Year'], ascending=False, inplace=True)
df_copy.head(5)

It is also possible to sort by multiple columns at once:

(Notice how the `ascending` kwarg takes a list that applies to the list of columns to sort on in corresponding order.)

In [ ]:
df_copy.sort_values(['Year', 'Title'], ascending=[False, True]).head(10)

Notice too that since we didn't use `inplace=True` in the previous example, our `DataFrame` remains sorted as before:

In [ ]:
df_copy.head(10)

#### 4.2.2 Activity

Given the `DataFrame` `df` defined above, write an expression to extract a `DataFrame` with the columns `Title`, `Year`, `imdbRating`, and `imdbVotes`. Show the first 5 rows in order of highest rating and then most votes.

In [ ]:
# Write your code here

In [ ]:
#@title → Double-click Here to Show/Hide a Prepared Solution
df[['Title','Year','imdbRating','imdbVotes']]\
    .sort_values(['imdbRating', 'imdbVotes'], ascending=[False, False])\
    .head(5)






### 4.3. Operations, filtering, and assignment

Operations performed on a column, or `Series`, are broadcast to each of the elements.

In [ ]:
# convert the running times to hours
df.Runtime / 60

Simple string concatenation can be performed in the same manner:

In [ ]:
df.Title + '(' + df.Rated + '), directed by: ' + df.Director

as can boolean operations:

In [ ]:
df.Year < 2000

By itself this is not terribly useful, but expressions of this kind are very powerful when passed to a `DataFrame` to select content:

In [ ]:
df[df.Production == 'Paramount Pictures']

In [ ]:
df[df.Year < 2000]

Any expression that evaluates to a `Series` of boolean values (`True` or `False`) and shares the same `Index` as the source `DataFrame` can be used.  Complex conditions can be assembled using bitwise logical operators `&`, `|`, and `~` to create simple but powerful filters.

In [ ]:
# returns a `Series`
df[(df.Year < 2000) & (df.imdbRating > 8)].Title

In [ ]:
# returns a `DataFrame`
df.loc[(df.Year < 2000) & (df.imdbRating > 8), ['Title', 'Year', 'imdbRating']]

The `.str` property gives access to string variables in a broadcast fashion, such that they can be manipulated:

In [ ]:
df.Actors.str.split(', ')

We can make use of `.str` in expressions:

In [ ]:
# select records with Oscar nominations or wins
df[df.Awards.str.contains('Oscar') == True]

In [ ]:
df[df.Director.apply(lambda a: len(a.split(', ')) > 3)][['Title', 'Director']]

#### 4.3.1. Assignment to `DataFrame`s

It is also possible to assign values directly to columns or cells in a data frame.

In [ ]:
df_copy = df.copy()

# create a new column based on an existing one
df_copy['Runtime (hrs)'] = df_copy.Runtime / 60

# edit a single value
df_copy.at['tt0111161', 'Title'] = 'Rita Hayworth and Shawshank Redemption'

df_copy[['Title', 'Runtime', 'Runtime (hrs)']]

#### 4.3.2. Activity

Returning to our “Bay Area Counties” data from earlier, write an expression to calculate the population density of each county, and assign it to a new column on the `DataFrame`.

In [ ]:
data = {
    "county": ["Alameda", "Contra Costa", "Marin", "Napa", "San Francisco", "San Mateo", "Santa Clara", "Solano", "Sonoma"],
    "county seat": ["Oakland", "Martinez", "San Rafael", "Napa", "San Francisco", "Redwood City", "San Jose", "Fairfield", "Santa Rosa"],
    "population": [1494876, 1037817, 250666, 135377, 870887, 711622, 1762754, 411620, 478551],
    "area": [2130, 2080, 2140, 2040, 600.59, 1930, 3380, 2350, 4580]  # data is in km²
}
bay_area_counties = pd.DataFrame(data)
bay_area_counties

In [ ]:
# Write your code here


In [ ]:
#@title → Double-click Here to Show/Hide a Prepared Solution
bay_area_counties['pop. density'] = bay_area_counties.population / bay_area_counties.area

### 4.4. Cleaning and manipulating `DataFrame`s

#### 4.4.1. Converting data types

Pandas does a pretty good job of inferring data types when we load data into a `DataFrame`, but sometimes we want or need to change the types it selects.  We can do this using the `.astype()` method on a `Series` object.

Those coming from an `R` or a statistical background may be used to working with categorical data, and we can force pandas to treat a column as categorical using `.astype('category')`:

In [ ]:
df['Rated'] = df['Rated'].astype('category')
df['Rated']

Using `.astype('category')` invokes the default behavior, by which categories are inferred from the data and are unordered.  The only real advantage to using categories is if they are ordered, so let's quickly convert them to ordered categories:

In [ ]:
from pandas.api.types import CategoricalDtype
df['Rated'] = df['Rated'].astype(CategoricalDtype(ordered=True))
df['Rated']

A common need is to parse dates -- if loaded from, e.g., a CSV file, they will often be interpreted as strings, as is the case with our “Released” column.

In [ ]:
df['Released'].astype('datetime64')

In [ ]:
pd.to_datetime(df['Released'])

In [ ]:
df['Released'].apply(pd.to_datetime)

In [ ]:
df[['Released']]

#### 4.4.2. `.apply()`

The `apply()` method has appeared a couple of times above. This is the fundamental way of manipulating the contents of `DataFrame`s.  `apply()` takes a function as an argument, and `apply`s the function to each element in the container it’s called on.

In [ ]:
def count_genres_naive(text):
    genres = text.split(", ")
    return len(genres)

In [ ]:
df.Genre

In [ ]:
# Note: This won't work...
df.Genre.apply(count_genres_naive)

For the purposes of this workshop, the version of the dataset we are currently working with has had some data deliberately removed, and unfortunately our `count_genres_naive` function does not know how to handle missing data.

Pandas uses the `NaN` datatype from the underlying `numpy` package to represent missing data, and this datatype is based on the primitive `float` type, which is why the `Attribute` error reads as it does.

In [ ]:
df[pd.isna(df.Genre)][['Title', 'Genre']]

We could handle this problem in a number of ways:
1. we could drop the observations with missing values immediately (and temporarily) before we apply our count function;
2. we could modify our `count_genres_naive` function to handle the missing values appropriately;
3. we could fix the data manually by assigning values where they are missing; or
4. we could clean the dataset when we initially import it.

In [ ]:
print(len(df.Genre),len(df.Genre.dropna()))

In [ ]:
df.Genre.dropna().apply(count_genres_naive)

One possible problem with this approach is that we don't get a count for the observations with missing values.  This means that if we want to enrich our original `DataFrame` with a new column containing the genre count, our count column will also be populated with `NaN`s:

In [ ]:
df['Genre Count'] = df.Genre.dropna().apply(count_genres_naive)
df[pd.isna(df.Genre)][['Genre', 'Genre Count']]

In this case, we might conclude that since the data is missing, the appropriate value for “genre count” is `0`, and we can write a less naïve operator function that handles this for us:

In [ ]:
def count_genres(text):
    if pd.isna(text):
        return 0
    genres = text.split(", ")
    return len(genres)

In [ ]:
df['Genre Count'] = df.Genre.apply(count_genres)
df[pd.isna(df.Genre)][['Genre', 'Genre Count']]

For the sake of our workshop, however, the dataset has been modified so that `Genre` has been removed where it should be the single value “Comedy”.  Since we know, therefore, what the value should be for all observations where `Genre` is missing, we have the option to simply update the `DataFrame` directly in the following way:

In [ ]:
df.loc[pd.isna(df.Genre), 'Genre'] = 'Comedy'
df[pd.isna(df.Genre)]

We might be done at this point, but it might also be a good idea to check if there are other missing values in our dataset:

In [ ]:
df.isnull().sum()

#### 4.4.3. Manipulating / cleaning on import

With this in mind, then, and especially when creating re-usable scripts for repetitive tasks or processes we want to iterate over, a common ‘best practice’ is to handle many data transformations when the `DataFrame` is first populated.

In [ ]:
from pandas.api.types import CategoricalDtype

column_types = {
    'Rated': CategoricalDtype(ordered=True)
}

fill_nans = {
    'Genre': 'Comedy',
    'imdbVotes': 0,
    'Rated': 'Unrated',
    'Awards': '',
    'Metascore': '',
    'BoxOffice': '',
    'Production': '',
    'RottenTomatoes': ''
}

date_columns = ['Released']

df = pd.read_csv(
    'workshop_data/imdb_top_1000.csv',
    parse_dates=date_columns,
    index_col='imdbID',
    dtype=column_types
)

df.fillna(value=fill_nans, inplace=True)

df.head(5)

In [ ]:
df.isnull().sum()

#### 4.4.4. Activity

Add a new column `language_count` to the `DataFrame` and show movies with more than 1 language.

In [ ]:
# Write your code here

In [ ]:
#@title → Double-click Here to Show/Hide Hints

# 1. You'll probably want to write a helper
#    function to count the languages

# 2. Create a new column to the DataFrame by 
#    apply()-ing the helper function, and add
#    it to the dataframe

# 3. Filter the dataframe based on the contents
#    of the new column

In [ ]:
#@title → Double-click Here to Show/Hide a Prepared Solution

def count_languages(langs):
    return len(langs.split(', '))
    
df['language_count'] = df.Language.apply(count_languages)
df[df.language_count > 1]

### 4.5. Aggregating data

But what about the most-featured Director in the top 1000 list?  Or the average rating for movies classified as “Comedy”?  For these kinds of operations we need to compute across aggregations of the dataset.  In Pandas this may be accomplished by means of `.groupby()` operation followed by an `.aggregate()` function. Aggregates can be considered a form of `.apply()` that operates on a collection of observations at once (in these cases, on the collections created by `.groupby()`.

<img src="https://raw.githubusercontent.com/sul-cidr/Workshops/master/Data_Manipulation_with_Python/assets/aggregation.png" alt="Aggregation">

Those who are familiar with the `R` programming language, may recognize this as the “Split-Apply-Combine” approach:[<sup>*</sup>](#fn)
- *Split* a dataset into relevant subsets;
- *Apply* a function to each subset;
- *Combine* all the pieces back together.

<hr>
<span id="fn">* A classic paper on the Split-Apply-Combine methodology is available here: <a href="https://www.jstatsoft.org/article/view/v040i01/v40i01.pdf">“The Split-Apply-Combine Strategy for Data Analysis”</a></span>

In [ ]:
df.groupby('Director')

In [ ]:
df.groupby('Director')[['Title']]

In [ ]:
df.groupby('Director')[['Title']].aggregate(len)

We can then sort on the aggregated values to show the highest (and lowest) ranking Directors among our top 1,000 movies:

In [ ]:
df.groupby('Director')[['Title']].aggregate(len).sort_values(['Title'], ascending=False).head(10)

If we want to aggregate across the combination of, e.g., Director and Production credit, the `.group_by()` method will accept a list of columns:

In [ ]:
(df.groupby(['Director', 'Production'])[['Title']]
   .aggregate(len)
   .sort_values(['Title'], ascending=False)
   .head(10))

It also is possible to perform this grouping, modification and presentation (and to generate other summaries of `DataFrame` contents) via the spreadsheet-style `pivot_table()` method.

In [ ]:
df.pivot_table(
    index=['Director', 'Production'],
    values=['Title'],
    aggfunc=len
).sort_values('Title', ascending=False).head(10)

### 4.5.1. Activity

Show the top five years for movies categorized as "Comedy".

(Hint: don't try to parse-out the different values in the “Genre” column -- just test for the presence of the word “Comedy”, e.g. `df.Genre.str.contains('Comedy')`.)

In [ ]:
# Write your code here

In [ ]:
#@title → Double-click Here to Show/Hide a Prepared Solution
df[df.Genre.str.contains('Comedy')].groupby('Year')[['Title']].aggregate(len).sort_values(['Title'], ascending=False).head(5)

## 5. Visualization

Pandas also provides some utilities to create basic plots just by calling `plot()` on a `Series` or `DataFrame`. But first we need to tell Jupyter that we are going to plot some charts using the plotting library `matplotlib`.

In [ ]:
# enables inline plotting in Jupyter using matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
df.groupby('Year')[['Year']].aggregate(len).plot()

Each time you call `plot()` an `AxesSubplot` object is returned, and these are automatically rendered by the Jupyter notebook environment. `AxesSubplot` objects are objects of the underlying `matplotlib` library for plotting in Python, and as such, lots of different options can be passed to control the output.

In [ ]:
ax = df.groupby('Year')[['Year']].aggregate(len).plot(
    kind="bar",
    figsize=(15, 5),
    title="# Movies per Year",
    legend=None
)
ax.set_ylabel("# Movies")
ax.set_xlabel("Year of Release")

In [ ]:
plt.style.available

In [ ]:
with plt.style.context('ggplot'):
    df.groupby('Year')[['Year']].aggregate(len).plot()

In [ ]:
with plt.xkcd():
    df.groupby('Year')[['Year']].aggregate(len).plot()
plt.rcdefaults()  # this is needed as the XKCD style is a special case

In [ ]:
df.groupby('Year')[['Year']].aggregate(len).plot()

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 5))
plt.ticklabel_format(style='plain')
df['imdbVotes'].hist(ax=ax, bins=15, density=True, color='lightseagreen')
df['imdbVotes'].plot(ax=ax, kind='kde', xlim=(0, 2200000), style='r--')
ax.set_title('Histogram of # votes')

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 6))
plt.ticklabel_format(style='plain')
df.boxplot(column='imdbVotes', by='Rated', grid=False, ax=ax, sym='')

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(1, figsize=(6, 6))
plt.ticklabel_format(style='plain')
sns.violinplot(y=df['imdbVotes'], grid=False, ax=ax)

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 5))
plt.ticklabel_format(style='plain')
ax.scatter(x=df.Year, y=df.imdbVotes)
ax.set_ylabel('# Votes')
ax.set_xlabel('Year')
ax.set_title('# Votes by Year')

## Evaluation survey
At the end of the workshop, we would be very grateful if you can, please, spend 1 minute answering a few questions that will help us to continue our workshop series.
- https://evaluations.cidr.link/Python_and_Pandas/